# Chain of Verification(CoVe)

LLMの出力からハルシネーション(事実と異なる間違い)を減らすための手法です．
- 論文:https://arxiv.org/abs/2309.11495

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/develop/nlp/llm_prompt_engineering/chain_of_verification.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install openai

In [ ]:
import os
from openai import OpenAI

#os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

client = OpenAI()

### Chain of Verification

In [ ]:
prompt1 = """
Q: 質問1
A: 1の答え
Q: 質問2
A: 2の答え
Q: 質問3
A:
"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt1}]
    )
output1 = response.choices[0].message.content
print(response.choices[0].message.content)

In [ ]:
prompt2 = """
Context: Q:質問1
A:1の答え
Response:
1の答えに含まれている事実，それを検証するための質問
1の答えに含まれている事実，それを検証するための質問

Context: Q:質問2
A:2の答え
Response:
2の答えに含まれている事実，それを検証するための質問
2の答えに含まれている事実，それを検証するための質問

Context: Q:質問3
A:{output1_}
Response:
""".format(output1=output1)

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt2}]
    )
response.choices[0].message.content
print(response.choices[0].message.content)

In [ ]:
prompt3_1 = """
Q:検証するための質問
A:答え

Q:検証するための質問
A:答え

Q:先ほど生成した質問
A:
"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt3_1}]
    )
output3_1 = response.choices[0].message.content
print(response.choices[0].message.content)

In [ ]:
prompt3_2 = """
Q:検証するための質問
A:答え

Q:検証するための質問
A:答え

Q:先ほど生成した質問
A:
"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt3_2}]
    )
output3_2 = response.choices[0].message.content
print(response.choices[0].message.content)

In [ ]:
prompt4 = """
Context:修正前の文章
別のソースからの情報
検証ステップの実行結果:Q + A
検証ステップの実行結果:Q + A
Response:修正された一貫性のある文章

Context:修正前の文章
別のソースからの情報
検証ステップの実行結果:Q + A
検証ステップの実行結果:Q + A
Response:修正された一貫性のある文章

Context:prompt1の答え
別のソースからの情報
検証ステップの実行結果:Q(prompt2の質問) + A:{output3_1_}
検証ステップの実行結果:Q(prompt2の質問) + A:{output3_2_}
Response:
""".format(output3_1_=output3_1, output3_2_=output3_2)


In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt4}]
    )

print(response.choices[0].message.content)